In [35]:
from PIL import Image
import numpy as np
import random
import math


In [45]:
# 読み込み
image_path = './dice_samples/sample_dice1_1.png'
original_image = Image.open(image_path)

# ダウンスケーリング
downscaled_size = (600, 600)
downscaled_image = original_image.resize(downscaled_size).convert('RGBA')


In [46]:
print(original_image.mode)


RGBA


In [42]:
# 回転行列を用いて各頂点の座標を計算
def rotate_point(x, y, theta, cx, cy):
    x -= cx
    y -= cy
    x_new = x * math.cos(theta) - y * math.sin(theta)
    y_new = x * math.sin(theta) + y * math.cos(theta)
    return x_new + cx, y_new + cy


def calculate_corners_cor(downscaled_size, random_angle, random_x_rotated, random_y_rotated):
    # 回転角をラジアンに変換
    theta = math.radians(random_angle)
    # 画像の中心を基準とする
    cx, cy = downscaled_size[0] / 2, downscaled_size[1] / 2
    # 元の画像の四隅の座標
    corners = [(0, 0), (downscaled_size[0], 0), (0, downscaled_size[1]), downscaled_size]
    # 回転後の座標を計算し、画像の位置を加算
    rotated_corners = [rotate_point(x, y, theta, cx, cy) for x, y in corners]
    translated_rotated_corners = [(x + random_x_rotated, y + random_y_rotated) for x, y in rotated_corners]
    
    return translated_rotated_corners


def paste_rotated_dice(background, background_size, downscaled_image):
    # ランダムな角度で画像を回転
    random_angle = random.randint(0, 360)
    rotated_image = downscaled_image.rotate(random_angle, expand=True)
    # 回転後の画像サイズを取得
    rotated_size = rotated_image.size
    # 回転後の画像が背景内に収まるように座標を制限する
    max_x_rotated = background_size[0] - rotated_size[0]
    max_y_rotated = background_size[1] - rotated_size[1]
    # 座標が負にならないようにする
    max_x_rotated = max(max_x_rotated, 0)
    max_y_rotated = max(max_y_rotated, 0)
    # 回転後の画像を配置するランダムな位置を決定
    random_x_rotated = random.randint(0, max_x_rotated)
    random_y_rotated = random.randint(0, max_y_rotated)
    # 回転した画像を背景に合成
    print(rotated_image.mode, background.mode)
    background.paste(rotated_image, (random_x_rotated, random_y_rotated))

    return background, random_angle, random_x_rotated, random_y_rotated


In [43]:
print(background)


<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x1EB21534D90>


In [44]:
# 背景画像生成
background_size = (2000, 2000)
background = Image.new("RGBA", background_size, (0, 0, 0, 0))

background, random_angle1, random_x_rotated1, random_y_rotated1 = paste_rotated_dice(background, background_size, downscaled_image)
background, random_angle2, random_x_rotated2, random_y_rotated2 = paste_rotated_dice(background, background_size, downscaled_image)

translated_rotated_corners = calculate_corners_cor(downscaled_size, random_angle1, random_x_rotated1, random_x_rotated1)


# 合成された画像をグレースケールに変換
grayscale_image_rotated = background.convert("L")

# グレースケール画像を表示
grayscale_image_rotated.show()

# グレースケール画像のパス
grayscale_image_path = './output/dice1_1_img/grayscale_image.png'
grayscale_image_rotated.save(grayscale_image_path)


RGBA RGBA
RGBA RGBA


In [ ]:


# 回転角をラジアンに変換
theta = math.radians(random_angle)

# 画像の中心を基準とする
cx, cy = downscaled_size[0] / 2, downscaled_size[1] / 2



# 元の画像の四隅の座標
corners = [(0, 0), (downscaled_size[0], 0), (0, downscaled_size[1]), downscaled_size]

# 回転後の座標を計算し、画像の位置を加算
rotated_corners = [rotate_point(x, y, theta, cx, cy) for x, y in corners]
translated_rotated_corners = [(x + random_x_rotated, y + random_y_rotated) for x, y in rotated_corners]

translated_rotated_corners



In [ ]:
def orientation(p, q, r):
    """Return the orientation of the triplet (p, q, r)
       0 --> p, q and r are collinear
       1 --> Clockwise
       2 --> Counterclockwise
    """
    val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
    if val == 0:
        return 0  # collinear
    return 1 if val > 0 else 2  # clock or counterclock wise

def do_lines_intersect(p1, q1, p2, q2):
    """Returns True if the line segments 'p1q1' and 'p2q2' intersect."""
    # Find the four orientations needed for general and special cases
    o1 = orientation(p1, q1, p2)
    o2 = orientation(p1, q1, q2)
    o3 = orientation(p2, q2, p1)
    o4 = orientation(p2, q2, q1)

    # General case
    if o1 != o2 and o3 != o4:
        return True

    # Special Cases
    # p1, q1 and p2 are collinear and p2 lies on segment p1q1
    if o1 == 0 and on_segment(p1, p2, q1):
        return True

    # p1, q1 and p2 are collinear and q2 lies on segment p1q1
    if o2 == 0 and on_segment(p1, q2, q1):
        return True

    # p2, q2 and p1 are collinear and p1 lies on segment p2q2
    if o3 == 0 and on_segment(p2, p1, q2):
        return True

    # p2, q2 and q1 are collinear and q1 lies on segment p2q2
    if o4 == 0 and on_segment(p2, q1, q2):
        return True

    return False  # Doesn't fall in any of the above cases

def on_segment(p, q, r):
    """Given three collinear points p, q, and r, check if point q lies on line segment 'pr'"""
    return (q[0] <= max(p[0], r[0]) and q[0] >= min(p[0], r[0]) and
            q[1] <= max(p[1], r[1]) and q[1] >= min(p[1], r[1]))

# Define points for two line segments: AB and CD
A = (1, 1)
B = (4, 4)
C = (1, 8)
D = (2, 2)

# Check if line AB intersects with line CD
do_lines_intersect(A, B, C, D)


In [ ]:
print(rotated_size)


In [ ]:
for idx_dice1 in range(1, 7):
    for idx_dice2 in range(idx_dice1, 7):
        continue


In [ ]:
# 読み込み
image_path = './dice_samples/sample_dice1.png'
original_image = Image.open(image_path)

# ダウンスケーリング
downscaled_size = (800, 800)
downscaled_image = original_image.resize(downscaled_size)


In [ ]:
# 背景画像生成
background_size = (2000, 2000)
background = Image.new("RGB", background_size, (0, 0, 0))

# ランダムな角度で画像を回転
random_angle = random.randint(0, 360)
rotated_image = downscaled_image.rotate(random_angle, expand=True)

# 回転後の画像サイズを取得
rotated_size = rotated_image.size

# 回転後の画像が背景内に収まるように座標を制限する
max_x_rotated = background_size[0] - rotated_size[0]
max_y_rotated = background_size[1] - rotated_size[1]

# 座標が負にならないようにする
max_x_rotated = max(max_x_rotated, 0)
max_y_rotated = max(max_y_rotated, 0)

# 回転後の画像を配置するランダムな位置を決定
random_x_rotated = random.randint(0, max_x_rotated)
random_y_rotated = random.randint(0, max_y_rotated)

# 新しい黒い背景画像を生成（前の画像は破棄）
background_rotated = Image.new("RGB", background_size, (0, 0, 0))

# 回転した画像を背景に合成
background_rotated.paste(rotated_image, (random_x_rotated, random_y_rotated))

# 合成された画像をグレースケールに変換
grayscale_image_rotated = background_rotated.convert("L")

# グレースケール画像を表示
grayscale_image_rotated.show()

# グレースケール画像のパス
grayscale_image_path = './output/dice1_1_img/grayscale_image.png'
grayscale_image_rotated.save(grayscale_image_path)
